In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import numpy as np


In [ ]:
raw_datasets=load_dataset("glue","mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_train_dataset=raw_datasets["train"]

In [ ]:
raw_train_dataset[1000]

{'sentence1': 'He took batting practice on the field for the second time Tuesday since his opening-day injury .',
 'sentence2': 'Jeter , who dislocated his left shoulder in a collision March 31 , took batting practice on the field for the first time Monday .',
 'label': 1,
 'idx': 1120}

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
token_s1=tokenizer(raw_train_dataset["sentence1"])
token_s2=tokenizer(raw_train_dataset["sentence2"])

In [ ]:
trial=tokenizer("This is first sentence.","This is second sentence. ")
trial

{'input_ids': [101, 2023, 2003, 2034, 6251, 1012, 102, 2023, 2003, 2117, 6251, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_fn(dataset):
    return tokenizer(dataset["sentence1"],dataset["sentence2"],truncation=True)

In [ ]:
tokenized_datasets=raw_datasets.map(tokenize_fn,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
collator=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

In [ ]:
sample=tokenized_datasets["train"][:10]

In [ ]:
sample={k:v for k,v in sample.items() if k not in ["idx","sentence1","sentence2"]}

In [ ]:
[len(x) for x in sample["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32, 45, 60]

In [ ]:
batch=collator(sample)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
{k:v.shape for k,v in batch.items()}

{'input_ids': TensorShape([10, 67]),
 'token_type_ids': TensorShape([10, 67]),
 'attention_mask': TensorShape([10, 67]),
 'labels': TensorShape([10])}

In [ ]:
tf_train_dataset=tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask","input_ids","token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=collator,
    batch_size=8
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
tf_validation_dataset=tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask","input_ids","token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=collator,
    batch_size=8
)

In [ ]:
for row in tf_validation_dataset:
  print(row)


Streaming output truncated to the last 5000 lines.
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(8, 65), dtype=int64, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 

In [ ]:
from transformers import TFAutoModelForSequenceClassification

In [ ]:
model=TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
model.compile(optimizer="adam",loss=SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

In [ ]:
model.fit(tf_train_dataset,validation_data=tf_validation_dataset)

459/459 [==============================] - 137s 188ms/step - loss: 0.6691 - accuracy: 0.6418 - val_loss: 0.6679 - val_accuracy: 0.6838


In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [ ]:
batch_size=8

In [ ]:
num_epochs=8

In [ ]:
num_train_steps=len(tf_train_dataset)*num_epochs

In [ ]:
lr_scheduler=PolynomialDecay(initial_learning_rate=5e-5,end_learning_rate=0.0,decay_steps=num_train_steps)

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
opt=Adam(learning_rate=lr_scheduler)

In [ ]:
model=TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow
loss = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=opt,loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(tf_train_dataset,validation_data=tf_validation_dataset,epochs=5)

Epoch 1/5
459/459 [==============================] - 130s 188ms/step - loss: 0.5347 - accuracy: 0.7274 - val_loss: 0.4322 - val_accuracy: 0.7990
Epoch 2/5
459/459 [==============================] - 80s 175ms/step - loss: 0.2990 - accuracy: 0.8795 - val_loss: 0.3806 - val_accuracy: 0.8505
Epoch 3/5
459/459 [==============================] - 77s 167ms/step - loss: 0.1269 - accuracy: 0.9594 - val_loss: 0.5509 - val_accuracy: 0.8407
Epoch 4/5
459/459 [==============================] - 76s 165ms/step - loss: 0.0639 - accuracy: 0.9796 - val_loss: 0.6227 - val_accuracy: 0.8333
Epoch 5/5
459/459 [==============================] - 73s 160ms/step - loss: 0.0318 - accuracy: 0.9905 - val_loss: 0.6056 - val_accuracy: 0.8627


In [ ]:
model.save("FirstMRPC_proper")

In [ ]:
preds=model.predict(tf_validation_dataset)["logits"]

51/51 [==============================] - 7s 53ms/step


In [ ]:
class_preds=np.argmax(preds,axis=1)

In [ ]:
print(preds.shape,class_preds.shape)

(408, 2) (408,)


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
metric=evaluate.load("glue","mrpc")

In [ ]:
metric.compute(predictions=class_preds,references=raw_datasets["validation"]["label"])

{'accuracy': 0.8627450980392157, 'f1': 0.9047619047619047}

In [ ]:
!zip -r /content/FirstMRPC_proper1.zip /content/FirstMRPC_proper

  adding: content/FirstMRPC_proper/ (stored 0%)
  adding: content/FirstMRPC_proper/variables/ (stored 0%)
  adding: content/FirstMRPC_proper/variables/variables.index (deflated 79%)
  adding: content/FirstMRPC_proper/variables/variables.data-00000-of-00001 (deflated 16%)
  adding: content/FirstMRPC_proper/saved_model.pb (deflated 92%)
  adding: content/FirstMRPC_proper/assets/ (stored 0%)
  adding: content/FirstMRPC_proper/fingerprint.pb (stored 0%)
  adding: content/FirstMRPC_proper/keras_metadata.pb (deflated 96%)


In [ ]:
from google.colab import files
files.download("/content/FirstMRPC_proper1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>